In [1]:
import ee
import xee
import geemap

import xarray as xr

# import eextract

import time
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:1081'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:1081'

In [2]:
ee.Authenticate()
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com', project='ee-xieyx')

# 01 8-day ET

In [4]:
def cal_ET(img):
  time_start = ee.Date(img.get('system:time_start'))

  Es = img.select('Es')
  Ei = img.select('Ei')
  Ec = img.select('Ec')

  ET = Ec.expression('Es + Ei + Ec', {
    "Es": Es, "Ei": Ei, "Ec": Ec
  }).rename('ET')

  return ET.copyProperties(img, img.propertyNames()) \
    .set('date', time_start.format('yyyy-MM-dd'))

ic = ee.ImageCollection("CAS/IGSNRR/PML/V2_v018").map(cal_ET)

In [5]:
ic.first()

In [7]:
ic = ic.filterDate(ee.Date('2020-01-01'), ee.Date('2022-01-01'))

In [8]:
ds = xr.open_dataset(
  ic, engine='ee',
  crs = 'EPSG:4326', 
  scale=0.5,
  geometry = ee.Geometry.Rectangle(70, 15, 140, 55)
)

ds.to_netcdf('D:/Data/PML_V2_V018_ET_050deg_8day_China_2020-2021.nc')

# 02 DEM

In [3]:
ic = ee.ImageCollection([ee.Image("USGS/SRTMGL1_003")])
ic

In [5]:
ds = xr.open_dataset(
  ic, engine='ee',
  crs = 'EPSG:4326', 
  # scale=0.002083333, # 250 m
  # scale=0.008333333, # 1 km
  scale=0.01, # 0.01deg
  geometry = ee.Geometry.Rectangle(110, 35, 140, 60)
)

ds.to_netcdf('E:/var_Elevation/USGS_SRTM_GL1_003_NorthEastern_China_0.01deg.nc')

d:\Softwares\Python\Lib\site-packages\xee\ext.py:688: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(
